In [29]:
import PyPDF2

In [30]:
def pdf_to_text(file):
    with open(file, "rb") as pdf:
        reader = PyPDF2.PdfReader(pdf, strict=False)
        pdf_text = []
        for page in reader.pages:
            content = page.extract_text()
            pdf_text.append(content)
        return ' '.join(pdf_text)

'\n        You are an advanced and sophisticated Interviewer and Recruiter powered with an AI model trained to optimize resumes based on a specific job description. \n        Your task is to enhance the user\'s existing resume by incorporating relevant keywords from the job description, valuing their past jobs or projects, and ensuring it aligns with the job requirements. \n        However, it\'s crucial not to over-qualify the user.\n        The user\'s existing resume content and The job description the user is applying to.\n        The resume is to be generated in the language below:\n        English\n        \n        Below are some tips to follow:\n        Tip: Match the skills in your resume to the exact spelling in the job description. Prioritize skills that appear most frequently in the job description.\n        Tip: Prioritize hard skills in your resume to get interviews, and then showcase your soft skills in the interview to get jobs.\n        Tip: Other keywords are words in

In [43]:
def create_prompt(language, tone):
    template = f"""
        You are an advanced and sophisticated Interviewer and Recruiter powered with an AI model trained to optimize resumes based on a specific job description. 
        Your task is to enhance the user's existing resume by incorporating relevant keywords from the job description, valuing their past jobs or projects, and ensuring it aligns with the job requirements. 
        However, it's crucial not to over-qualify the user.
        The user's existing resume content and The job description the user is applying to.
        The resume is to be generated in the language below:
        {language}
        
        Below are some tips to follow:
        Tip: Match the skills in your resume to the exact spelling in the job description. Prioritize skills that appear most frequently in the job description.
        Tip: Prioritize hard skills in your resume to get interviews, and then showcase your soft skills in the interview to get jobs.
        Tip: Other keywords are words included in the job description more than 3 times and not hard skills or soft skills. These words are typically buzzwords, industry lingo, or company specific jargon that may be unique to the specific company and help your resume get noticed. 
        Other keywords have a low impact on your match score. Spend less time including these in your resume on a case-by-case basis.
        Tip:  Consider adding at least 5 specific achievements or impact you had in your job (e.g. time saved, increase in sales, etc).
        Tip: Avoid negative phrase or cliche in the resume.
        Tip: Position yourself as an expert in your field. This can help to set you apart from other candidates who may not be as confident in their abilities.
        Tip: Use attention-grabbing action verbs. The example above uses the verb “leading,” which quickly tells employers what the applicant has accomplished.
        Tip: Be specific. Generic phrases such as “hard worker” or “team player” are nice, but they don’t really tell employers anything. If you can, include a specific accomplishment or skill that makes you stand out from the rest.
        Tip: Keep it brief – no more than a few sentences or bullet points.
        Tip: Look for patterns in your work history – anything that you can point to and say “this is what I do, and I’m good at it.”
        Tip: Focus only on your most relevant skills and experience.
        Tip: Use numbers and specifics to show that you are a results-oriented individual who is able to produce tangible outcomes.
        Tip: Incorporate keywords from the job description whenever possible.
        Tip: Tailor your summary statement to each job you apply for.
        Tip: Identify important keywords and skills. 
        Tip: Highlight achievements as well as responsibilities Instead of saying, “Managed a team of 12 people.” You could say, “Managed a team of 12 people, consistently meeting or exceeding quarterly targets.”
        Tip: Use action verbs, Use active voice, not passive eg: Active voice, “The chef prepared the meal.” Passive voice, “The meal was prepared by the chef.”
        Tip: Use numbers eg, Instead of saying, “Created marketing campaigns.” You could say, “Created 10 successful marketing campaigns that generated a 20% increase in leads.”
        Here are some more examples of using numbers (with action verbs underlined)
        Saved $7 million while introducing nationwide transport service for medical patients.
        Successfully increased sales by 20% within the first quarter of implementing a new marketing strategy.
        Streamlined project management process, reducing overall project completion time by 15% and improving team efficiency.
        
        Your Role as a Job Scanner
        Your task is to generate a revised resume that is tailored to the job description. 
        The output should be a well-structured JSON object containing the following sections: user name, profile description, email, LinkedIn, experiences (with details about the tasks completed and the period), education (in a structured way with the name of the college or university, diploma, and year), hard and soft skills, and hobbies.
        
        The structure of the JSON object should be as follows:

                {{
        "user_information": {{
            "name": "",
            "main_job_title": "",
            "profile_description": "",
            "email": "",
            "linkedin": "",
            "github": "",
            "experiences": [
            {{
                "job_title": "",
                "company": "",
                "start_date": "",
                "end_date": "",
                "four_tasks": []
            }},
            ...
            ],
            "education": [
            {{
                "institution": "",
                "degree": "",
                "start_date": "",
                "end_date": ""
            }},
            ...
            ],
            "skills": {{
                "hard_skills": [],
                "soft_skills": []
            }},
            "hobbies": []
        }},
        "projects": [{{
            "project_name": "",
            "two_goals_of_the_project": [],
            "project_end_result": ""
        }}
        ],
        "certificate": [{{
            "name": "",
            "institution": "",
            "description": "",
            "date": ""
        }}
        ],
        "extra_curricular_activities": [{{
            "name": "",
            "description": ""
        }}
        ],
        ...
        }}
        
        Generate a tailored resume that emphasizes relevant experiences, skills, and qualifications for the mentioned job. Ensure the output is in a well-structured JSON format.
        
        Please ensure that the tone of the resume matches the provided tone parameter: {tone}.
        'Professional' for formal settings, 'Creative' for showcasing creativity, 'Balanced' for a mix, and 'Expert' for highlighting specialized expertise.
        
        NB: You are to only reply the output (structured result) nothing else, Not even an inductory sentence to the output."""
    
    return template

def extract_json(text):
	start = text.find("{")
	end = text.rfind("}") + 1
	return text[start:end]

from octoai.text_gen import ChatMessage
from octoai.client import OctoAI

def generate_resume(template, resume_content, job_description, API_KEY):
    client = OctoAI(api_key=API_KEY,)
    
    completion = client.text_gen.create_chat_completion(
        max_tokens=10000,
        messages=[
            ChatMessage(
                content=f"{template}",
                role="system"
            ),
            ChatMessage(
                content=f"{resume_content}\n{job_description}",
                role="user"
            )
        ],
        model="mixtral-8x22b-instruct",
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9
    )

    return extract_json(completion.choices[0].message.content)

template = create_prompt("English", "Professional")
resume = pdf_to_text("../../../../Téléchargements/DOSSEH_SHALOM_CV.pdf")
job_description = """
About the job
Who We Are Is What We Do.

Deel and our family of growing companies are made up of global teams dedicated to helping businesses hire anyone, anywhere, easily.

The team comprises over three thousand self-driven individuals spanning over 100 countries, and our unified yet diverse culture keeps us continually learning and innovating the platform and products for customers.

Companies should be able to hire the best talent anywhere in the world, so we are building the best platform to make that a reality. Our market-leading technology, expertise, and global team are crucial to the platform’s success. We deliver the best products and features in our space, enabling millions of jobs worldwide and connecting the global workforce with the best companies and opportunities.

Why should you be part of our success story?

A 30-mile hiring radius should no longer dictate how companies hire because exceptional talent lives everywhere. Deel sees a world without hiring borders and endless talent that pairs perfect candidates with great companies.

We offer global teams all the tools they need to hire, onboard, manage, pay, and scale at full speed. We aim to foster a diverse global economy by building a generational platform that seamlessly connects companies with talent worldwide.

After our successful Series D in 2021, we raised another $50M in 2023, doubling our valuation to $12B. There’s never been a more exciting time to join Deel — the international payroll and compliance market leader.

Duties

Collaborate closely with Product Managers, Product Operations and Dev teams on different problems to create actionable insights
Monitor product and feature release performance, define baselines, success metrics and provide recommendations for optimisation
Develop/apply extensive domain knowledge in your dedicated product areas in order to proactively derive meaningful recommendations and insights that drive and inform product strategy
Analyse market data, identify customer behaviour and trends
Help design and evaluate A/B tests to improve customer journeys
Create and promote effective self-service analytics infrastructure for your product areas
Develop user friendly dashboards and high quality data visualisations that drive actions and support decision making
Explore large and complex datasets to create actionable insights
Build data models in dbt with the support of Analytics Engineers to create the cleanest data structures and KPIs for your product areas
Make intelligent choices for the adoption of new technologies based on your extensive experience
Contribute to the development and growth of fellow product analysts by proactively sharing knowledge, providing mentoring, etc.


Requirements

Bachelor Degree in Computer Science, Math, Statistics, Economics or equivalent degree
5+ years’ experience in product analytics team
Hands on experience in at least one modern BI tools (preferably Looker)
Proficiency in SQL and product analytics tools (mixpanel / heap / amplitude)
Solid experience with Data Modelling (DBT)
Excellent multitasking and problem-solving abilities
Effective communication skills and experience dealing with stakeholders of varying seniority


We’d especially love to speak with you if:

You love data exploration and visualisations
Have experience with snowflake and LookML
You are driven by business impact generated from your insights
You are naturally curious and enjoy challenging the status quo
You are proactive and like to take responsibility in projects, managing requirements and collaborating with a number of different teams across the company
You know when to go deep on a task and when to execute


Total Rewards

Our workforce deserves fair and competitive pay that meets them where they are. With scalable benefits, rewards, and perks, our total rewards programs reflect our commitment to inclusivity and access for all.

Some things you’ll enjoy

Provided computer equipment tailored to your role
Stock grant opportunities dependent on your role, employment status and location
Additional perks and benefits based on your employment status and country
The flexibility of remote work, including WeWork access where available


At Deel, we’re an equal-opportunity employer that values diversity and positively encourage applications from suitably qualified and eligible candidates regardless of race, religion, sex, national origin, gender, sexual orientation, age, marital status, veteran status, disability status, pregnancy or maternity or other applicable legally protected characteristics.
"""
API_KEY = "key"
result = generate_resume(template, resume, job_description, API_KEY)
print(result)

{
"user_information": {
    "name": "Shalom DOSSEH",
    "main_job_title": "Data Analyst",
    "profile_description": "Data Analyst/Business with a solid foundation in technical and business analysis. Proficient in advanced data analysis tools such as Excel, SQL, Python, R, and Tableau. Able to transform complex data into valuable business insights. Recognized for attention to detail and ability to identify business needs. Seeking to utilize these skills to facilitate data-driven decision-making.",
    "email": "shalom.dosseh@epitech.eu",
    "linkedin": "https://www.linkedin.com/in/shalom-dosseh-4a484a262/",
    "github": "https://github.com/AnalyticAce",
    "experiences": [
        {
            "job_title": "Data Analyst",
            "company": "Gozem Africa - First African Super App",
            "start_date": "September 2023",
            "end_date": "January 2024",
            "four_tasks": [
                "Drove decision-making through comprehensive data analysis.",
        